In [2]:
# Initialize PySpark path 
import findspark
findspark.init('/home/jing/spark-2.1.0-bin-hadoop2.7')

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
# Import data
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [6]:
my_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 10 rows



In [7]:
my_log_reg_model = LogisticRegression()

In [8]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [9]:
log_summary = fitted_logreg.summary

In [12]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [13]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [16]:
# Introduce to use evaluaters to show evaluation
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [17]:
final_model = LogisticRegression()

In [18]:
fit_final = final_model.fit(lr_train)    #fit on the training data 

In [19]:
prediction_and_labels = fit_final.evaluate(lr_test) #evaluate on the test data

In [20]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[22.0845094930485...|[0.99999999974365...|       0.0|
|  0.0|(692,[98,99,100,1...|[23.6983429409467...|[0.99999999994895...|       0.0|
|  0.0|(692,[121,122,123...|[24.6801526383058...|[0.99999999998087...|       0.0|
|  0.0|(692,[122,123,148...|[23.4771609223736...|[0.99999999993632...|       0.0|
|  0.0|(692,[123,124,125...|[23.9601655175250...|[0.99999999996071...|       0.0|
|  0.0|(692,[124,125,126...|[29.2510200542126...|[0.99999999999980...|       0.0|
|  0.0|(692,[124,125,126...|[21.1799891373738...|[0.99999999936664...|       0.0|
|  0.0|(692,[126,127,128...|[24.0688126845711...|[0.99999999996475...|       0.0|
|  0.0|(692,[126,127,128...|[14.3658964595689...|[0.99999942327312...|       0.0|
|  0.0|(692,[126

In [22]:
# import evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator()  

In [24]:
# since prediction_and_labels.predictions meet the default parameter names for the binary classification evaluator
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)  # evaluate the prediction

In [25]:
my_final_roc

1.0

In [27]:
# 1.0 implies the prediction was perfect and the data was completely separable